# S&P symbols

In [57]:
import sys
!{sys.executable} -m pip install pandas_datareader yfinance matplotlib

You should consider upgrading via the '/usr/local/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [58]:
import pandas as pd
from pandas import DataFrame
import json
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

yf.pdr_override()
# no need for requests or BeautifulSoup use read_html
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
# convert symbol column to list
def get_company_name(s):
    infos = df.loc[df['Symbol'] == s] 
    tickers = df['Symbol'].values.tolist()
    return infos.iloc[0]["Security"]

df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [59]:
# list comprehension to replace data in strings

ma_count = 21

data = pdr.get_data_yahoo(tickers, start=dt.datetime(2021,1,1), end=dt.date.today())['Close']
data.tail()

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-02,148.820007,21.480000,211.050003,139.960007,115.169998,116.620003,317.019989,118.040001,304.839996,593.070007,...,66.820000,144.559998,63.169998,63.889999,120.889999,116.949997,163.520004,539.940002,53.259998,192.699997
2021-07-06,148.979996,21.010000,208.699997,142.020004,115.730003,115.419998,326.709991,118.820000,305.399994,596.900024,...,67.070000,144.169998,61.369999,63.090000,120.459999,116.250000,162.779999,544.349976,51.720001,194.020004
2021-07-07,149.490005,20.309999,209.440002,144.570007,116.750000,115.040001,334.529999,119.870003,309.609985,605.770020,...,67.690002,137.440002,60.410000,63.630001,120.669998,117.820000,161.100006,546.650024,51.310001,197.399994
2021-07-08,148.830002,20.350000,207.509995,143.240005,116.349998,114.440002,325.040009,119.260002,309.149994,605.950012,...,67.500000,133.770004,60.139999,61.910000,119.190002,117.320000,159.259995,539.840027,49.840000,196.960007
2021-07-09,150.029999,20.889999,213.470001,145.110001,116.580002,115.129997,324.769989,119.739998,312.619995,604.500000,...,67.669998,135.589996,61.230000,62.419998,121.050003,118.580002,159.309998,547.729980,52.290001,197.500000


In [60]:
ma = data.rolling(window=ma_count).mean()
ma_pers = data.div(ma).mul(100).sub(100)
#ma_pers.iloc[[-1]]
ma_pers_r=ma_pers.iloc[::-1]
d = {}
df_prep = []
for col_name in ma_pers_r.columns:
    d[col_name]=0
    ma_last = None
    for index, row in ma_pers_r.iterrows():
        if row[col_name] > 0:
            d[col_name] += 1
            if ma_last == None:
                ma_last=row[col_name]
        else:
            if d[col_name] != 0:
               company = get_company_name(col_name)
               df_prep.append([col_name, company, d[col_name], ma_last, ma_count, "daily"]) 
            break
df = DataFrame(df_prep,columns=['ticker', 'name', 'count', 'ma_diff', 'ma_size', 'ma_type']).sort_values(by=['ma_diff', 'count'], ascending=False)
print(df['count'][df['count'] >= 10].size)
lead_comps_df = df[df['count'] >= 10]
lead_comps = lead_comps_df['ticker'].values.tolist()
lead_comps_df

141


,ticker,name,count,ma_diff,ma_size,ma_type
195,NKE,Nike,12,12.014842,21,daily
119,GNRC,Generac Holdings,32,10.202129,21,daily
160,KMX,CarMax,14,8.703945,21,daily
96,ENPH,Enphase Energy,35,8.081620,21,daily
199,NRG,NRG Energy,26,7.906564,21,daily
...,...,...,...,...,...,...
221,PKI,PerkinElmer,21,1.166190,21,daily
18,AMGN,Amgen,11,1.143987,21,daily
15,ALXN,Alexion Pharmaceuticals,65,0.946782,21,daily
38,BMY,Bristol-Myers Squibb,10,0.678161,21,daily


In [61]:
for quote in lead_comps:
    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(data[quote], label=quote);
    plt.plot(ma[quote], label = 'MA');
    plt.legend(loc=2);

KeyError: 'Symbol'